# Step 1: Data and Model Preparation


In [1]:
exp_name = 'exp_056'

In [2]:
%load_ext autoreload
%autoreload 2
import sys
import os
codebase = '../../'
sys.path.append(codebase)

In [3]:
from src.dataflow.vg_graph_loader_3 import VGGraphLoader
from src.dataflow.graph_batcher_4_rpn import GraphBatcher

In [4]:
from src.tensorflow.models.vsp_model_002_abl_05 import Model
from src.tensorflow.engine.trainer_3 import Trainer
from src.tensorflow.engine.util import deploy, load_params
from src.util.util import pickle_save, pickle_load
from src.eval.sgg_evaluator_04 import Evaluator


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [5]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(1234)
np.random.seed(1235)

In [6]:
gl = VGGraphLoader(
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'sg_stanford_2.pkl'),
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'word_emb_stanford_2.pkl'),
)


In [7]:
img_ids, train_idx, test_idx = pickle_load(os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'split_stanford.pkl'))
misc = pickle_load(os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'misc_stanford_2.pkl'))
img_no_ent = misc['img_no_ent']
img_no_pred = misc['img_no_pred']
image_size = misc['img_size']

In [8]:
train_idx = np.asarray(list(set(train_idx) - set(img_no_ent)))
np.random.shuffle(train_idx)
val_idx = train_idx[:1000]
train_idx = train_idx[1000:]

In [9]:
proposals = pickle_load(os.path.join(codebase, 'data', 'VG', 'xfeat_gtbox', 'iresnet_oi_lowprop', 'coords.pkl'))

In [10]:
batcher = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_gtbox', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1236, 
)
batcher.set_subset(train_idx)
batcher.shuffle()

In [11]:
batcher_val = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_gtbox', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1237, 
)
batcher_val.set_subset(val_idx)

In [12]:
tf.reset_default_graph()

In [13]:
model = Model(   
    dim_att_head_role_ent=[1024, 1024], 
    dim_att_head_role_pred=[1024, 1024],
    role_edge_mode='full_soft',
    stop_grad_role_edge=True,
    
    dim_state_ent=1024,
    dim_state_pred=1024,
    dim_emb_head_ent=[300],
    dim_conf_head_ent=[],
    dim_conf_head_pred=[],
    dim_emb_head_pred=[300],
    dim_init_head_ent=[1024],

    dim_message_send_head_pred2ent=[1024, 1024],
    dim_message_pool_head_pred2ent=[1024, 1024],
    dim_message_receive_head_pred2ent=[1024, 1024],    
    dim_message_send_head_ent2pred=[1024, 1024],
    dim_message_pool_head_ent2pred=[1024, 1024],
    dim_message_receive_head_ent2pred=[1024, 1024],
    
    num_mp_iter=3,     
    num_proposals=None,

    heat_role=1.0,
    heat_emb_mil=1.0,
    null_att_logit=0.0,
        
    num_align_iter=3,
    alignment_order='best',
    default_act_fn_name='leaky_relu',
    optimizer_type_generator=tf.train.AdamOptimizer,
    
    max_num_pred=100,
    dim_init_head_pred=[],
    init_state_type_pred='trainable',
    
    dim_proposal_feat=1536,
    emb_dict_ent=gl.noun_emb_dict,
    emb_dict_pred=gl.pred_emb_dict,    
    #image_size=448,
    #num_channels=3,
    debugging=False
)



Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
Instructions for updating:
Use tf.cast instead.


In [14]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True)))
#sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5)))
sess = tf.Session()
sess.run(tf.no_op())

# Step 2: Training


In [14]:
train_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': None,
    'filter_nonoverlap': True,
    'rank_triplet_confidence': False,
})
val_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': None,
    'filter_nonoverlap': True,
    'rank_triplet_confidence': False,
})


In [15]:
trainer = Trainer(
    model, sess, batcher, batcher_val, train_evaluator, val_evaluator,
    logdir=os.path.join(codebase, 'log', 'tfsummary', exp_name), 
    modeldir=os.path.join(codebase, 'checkpoint', exp_name),
)

In [ ]:
trainer.train (
    max_steps=30000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=10000, 
    hyperparams = {
        'learning_rate_generator': 1e-5 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 10.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


../../src/eval/eval_utils.py:55: RuntimeWarning: invalid value encountered in true_divide
  iou = intsec / (area1[:, np.newaxis] + area2[np.newaxis, :] - intsec)
../../src/eval/graph_util_new_02.py:118: RuntimeWarning: invalid value encountered in greater
  overlap = (pw_iou(ent_box[i], ent_box[i]) > 0.0).astype(np.float32)
../../src/eval/graph_eval_3.py:219: RuntimeWarning: invalid value encountered in true_divide
  per_image_recall.append(np.divide(len(gt_matched), len(gt_graph['pred_lbl'][i])))
../../src/eval/graph_eval_3.py:300: RuntimeWarning: invalid value encountered in true_divide
  per_image_recall.append(np.divide(len(gt_matched), len(gt_graph['pred_lbl'][i])))


Step 1000: loss = 0.8830000162124634 (1469.959 sec)
Training Batch Evaluation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.036, 'PerImageOneHopMR_at100_iou_0.5': 0.53, 'PerImageOneHopMP_at50_iou_0.5': 0.061, 'PerImageOneHopMR_at50_iou_0.5': 0.467, 'PerImageLocPredMP_at100_iou_0.5': 0.04, 'PerImageLocPredMR_at100_iou_0.5': 0.561, 'PerImageLocPredMP_at50_iou_0.5': 0.067, 'PerImageLocPredMR_at50_iou_0.5': 0.494}
Validation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.034, 'PerImageOneHopMR_at100_iou_0.5': 0.528, 'PerImageOneHopMP_at50_iou_0.5': 0.061, 'PerImageOneHopMR_at50_iou_0.5': 0.484, 'PerImageLocPredMP_at100_iou_0.5': 0.038, 'PerImageLocPredMR_at100_iou_0.5': 0.561, 'PerImageLocPredMP_at50_iou_0.5': 0.067, 'PerImageLocPredMR_at50_iou_0.5': 0.516}
Step 2000: loss = 0.8510000109672546 (2857.150 sec)
Training Batch Evaluation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.039, 'PerImageOneHopMR_at100_iou_0.5': 0.561, 'PerImageOneHopMP_at50_iou_0.5': 0.068, 'PerImageOneHopMR_at50_iou_

In [ ]:
trainer.train (
    max_steps=20000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=1000, 
    hyperparams = {
        'learning_rate_generator': 1e-6 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 10.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


In [18]:
trainer.train (
    max_steps=10000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=1000, 
    hyperparams = {
        'learning_rate_generator': 1e-6 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 10.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


Step 51000: loss = 0.3190000057220459 (1198.269 sec)
Training Batch Evaluation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.062, 'PerImageOneHopMR_at100_iou_0.5': 0.816, 'PerImageOneHopMP_at50_iou_0.5': 0.104, 'PerImageOneHopMR_at50_iou_0.5': 0.721, 'PerImageLocPredMP_at100_iou_0.5': 0.068, 'PerImageLocPredMR_at100_iou_0.5': 0.835, 'PerImageLocPredMP_at50_iou_0.5': 0.113, 'PerImageLocPredMR_at50_iou_0.5': 0.737}
Validation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.058, 'PerImageOneHopMR_at100_iou_0.5': 0.7, 'PerImageOneHopMP_at50_iou_0.5': 0.1, 'PerImageOneHopMR_at50_iou_0.5': 0.638, 'PerImageLocPredMP_at100_iou_0.5': 0.064, 'PerImageLocPredMR_at100_iou_0.5': 0.73, 'PerImageLocPredMP_at50_iou_0.5': 0.109, 'PerImageLocPredMR_at50_iou_0.5': 0.663}
Step 52000: loss = 0.2460000067949295 (2461.209 sec)
Training Batch Evaluation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.064, 'PerImageOneHopMR_at100_iou_0.5': 0.82, 'PerImageOneHopMP_at50_iou_0.5': 0.107, 'PerImageOneHopMR_at50_iou_0.

In [19]:
trainer.train (
    max_steps=10000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=1000, 
    hyperparams = {
        'learning_rate_generator': 1e-7 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 10.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


Step 61000: loss = 0.18700000643730164 (1172.990 sec)
Training Batch Evaluation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.067, 'PerImageOneHopMR_at100_iou_0.5': 0.844, 'PerImageOneHopMP_at50_iou_0.5': 0.111, 'PerImageOneHopMR_at50_iou_0.5': 0.736, 'PerImageLocPredMP_at100_iou_0.5': 0.073, 'PerImageLocPredMR_at100_iou_0.5': 0.859, 'PerImageLocPredMP_at50_iou_0.5': 0.12, 'PerImageLocPredMR_at50_iou_0.5': 0.752}
Validation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.06, 'PerImageOneHopMR_at100_iou_0.5': 0.707, 'PerImageOneHopMP_at50_iou_0.5': 0.102, 'PerImageOneHopMR_at50_iou_0.5': 0.645, 'PerImageLocPredMP_at100_iou_0.5': 0.065, 'PerImageLocPredMR_at100_iou_0.5': 0.734, 'PerImageLocPredMP_at50_iou_0.5': 0.11, 'PerImageLocPredMR_at50_iou_0.5': 0.668}
Step 62000: loss = 0.08699999749660492 (2275.504 sec)
Training Batch Evaluation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.068, 'PerImageOneHopMR_at100_iou_0.5': 0.867, 'PerImageOneHopMP_at50_iou_0.5': 0.113, 'PerImageOneHopMR_at50_i

In [21]:
trainer.save_model('final_1')

# Step 3: Evaluation

In [15]:
load_params(sess, os.path.join(codebase, 'checkpoint', exp_name, 'ckpt-70000'))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../../checkpoint/exp_056/ckpt-70000


In [16]:
gl2 = VGGraphLoader(
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'sg_stanford_with_duplicates.pkl'),
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'word_emb_stanford_2.pkl'),
)


In [17]:
test_evaluator = Evaluator({        
    'graph_loader': gl2,
    'proposals': proposals,
    'num_proposals': None,
    'filter_nonoverlap': True,
    'rank_triplet_confidence': False,
})


In [18]:
test_idx = np.asarray(list(set(test_idx) - set(img_no_ent)))

In [19]:
batcher_test = GraphBatcher(
    gl2, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_gtbox', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1237, 
)
batcher_test.set_subset(test_idx)

In [20]:
#test_evaluator.reset()
deploy(sess, model, batcher_test, test_evaluator, model.eval_ops,
    hyperparams={
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
    ignore_errors=False,
    debug_mode=False,
    verbose_rate=100
)


Deploying
Batch 1/997 done in 1 seconds. Spent time for read=0.01 (avg 0.01), processing=1.41 (avg 1.41), write=0.00 (avg 0.00) seconds.
Batch 101/997 done in 9 seconds. Spent time for read=0.01 (avg 0.01), processing=0.07 (avg 0.09), write=0.00 (avg 0.00) seconds.
Batch 201/997 done in 16 seconds. Spent time for read=0.01 (avg 0.01), processing=0.07 (avg 0.08), write=0.00 (avg 0.00) seconds.
Batch 301/997 done in 24 seconds. Spent time for read=0.00 (avg 0.01), processing=0.07 (avg 0.08), write=0.00 (avg 0.00) seconds.
Batch 401/997 done in 31 seconds. Spent time for read=0.00 (avg 0.01), processing=0.06 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 501/997 done in 39 seconds. Spent time for read=0.01 (avg 0.01), processing=0.08 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 601/997 done in 47 seconds. Spent time for read=0.01 (avg 0.01), processing=0.07 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 701/997 done in 55 seconds. Spent time for read=0.01 (avg 0.01), processing=0.

In [27]:
test_results = test_evaluator.evaluate()
test_results

{'PerImageOneHopMP_at100_iou_0.5': 0.05650301167022211,
 'PerImageOneHopMR_at100_iou_0.5': 0.7015302766704027,
 'PerImageOneHopMP_at50_iou_0.5': 0.0950163132137031,
 'PerImageOneHopMR_at50_iou_0.5': 0.6391304248350773,
 'PerImageLocPredMP_at100_iou_0.5': 0.06468346091103025,
 'PerImageLocPredMR_at100_iou_0.5': 0.737087689779692,
 'PerImageLocPredMP_at50_iou_0.5': 0.10801041535951815,
 'PerImageLocPredMR_at50_iou_0.5': 0.6735589373513056}